In [ ]:
import pyemma
import simplex
import mdshare
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pentapeptide_pdb = mdshare.fetch('pentapeptide-impl-solv.pdb')
pentapeptide_xtcs = mdshare.fetch('pentapeptide*xtc')

In [ ]:
feat = pyemma.coordinates.featurizer(topfile=pentapeptide_pdb)
feat.add_distances(feat.select_Heavy())
source = pyemma.coordinates.source(pentapeptide_xtcs, features=feat)

In [ ]:
tica = pyemma.coordinates.tica(source, lag=30)

In [ ]:
plt.plot(tica.timescales[0:15], 'o')

In [ ]:
ics = tica.get_output(dimensions=slice(0, 9))

In [ ]:
dims = range(2, 9)
score = []
for i in dims:
    vertices = simplex.find_vertices_inner_simplex(ics, take_max_dims=i)
    s = simplex.simplex_misfit(ics, vertices, extrema=False)
    score.append(s)
    print(s)

In [ ]:
plt.plot(dims, score)
plt.xlabel('# dimensions')
plt.ylabel('score')

In [ ]:
vertices = simplex.find_vertices_inner_simplex(ics, take_max_dims=6)
mems = simplex.memberships(ics, vertices)

In [ ]:
ctrajs = simplex.core_assignment_given_memberships(mems, f=0.75, d=0.1)

In [ ]:
simplex.scatter_mem(memberships=mems, ctrajs=ctrajs, selection=range(0, 7))

In [ ]:
ctrajs = simplex.filter_out_short_core_visits(ctrajs=ctrajs, cutoff=1)
dtrajs = simplex.assign_transitions_to_cores(ctrajs, keep_ends_unassigned=False)
#import msmtools
#C = msmtools.estimation.count_matrix(dtraj=dtrajs, lag=1).toarray()
#pyemma.plots.plot_network(C, arrow_label_format='%d')

In [ ]:
its = pyemma.msm.its(dtrajs=dtrajs, lags=100)
pyemma.plots.plot_implied_timescales(its, ylog=False)

In [ ]:
#for dtraj in dtrajs:
#    plt.figure()
#    plt.plot(dtraj)